# Is My Study Under-Powered?
__This notebook describes the power of 2-sample study with the help of ipywidgets.__
<br>The study assumes you are trying to make a change to __improve performance__ in a lognormal population.
<br>__("Improvement" implies a single, right-tailed test.)__ 

Lognormal is consistent with many production systems. Normal ditribution is inaccurate in
<br>these situations because the output does not make sense (a marketing decision can never make negative
<br>sales of a product occur).

There are 2 different approaches illustrated:
1. For an available sample size, what's the expected pvalue?
2. For a desired pvalue, how big of a sample size would be needed?

In both situations, expected impact of the change and variance of the population/sample are also assumed.

## 1. Significance level given test type, signal strength, variance, sample size

In [7]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import scipy.optimize

import ipywidgets as widgets
import IPython.display

In [21]:
percent_uplift_slider = widgets.FloatSlider(
    value = 0.03,
    min= 0.0,
    max= 1.0,
    step= 0.01,
    description = 'Expected Percent Uplift:',
    disabled = False,
    continuous_update = False,
    orientation ='horizontal',
    readout = True,
    readout_format = ".0%",
    style = {'description_width': 'initial'},
    layout=dict(width='40%')
)

st_dev_as_percent_of_mean_slider = widgets.FloatSlider(
    value = 0.2,
    min= 0.01,
    max= 2.0,
    step= 0.01,
    description = 'St Dev as % of Mean:',
    disabled = False,
    continuous_update = False,
    orientation ='horizontal',
    readout = True,
    readout_format = ".0%",
    style = {'description_width': 'initial'},
    layout=dict(width='60%')
)

sample_size_slider = widgets.SelectionSlider(
    options = [str(i) + " total samples" for i in range(4,52,2)],
    value = "4 total samples",
    description = 'Total number across both samples:',
    disabled = False,
    continuous_update = False,
    orientation ='horizontal',
    readout = True,
    style = {'description_width': 'initial'},
    layout=dict(width='80%')
)

In [22]:
def function_for_interact(effect_uplift, sample_stdev, sample_size):
    sample_size = int(sample_size.split()[0])
    
    ### Log-space parameters
    ### Using 1 as mu (basis point)
    H_null = 1
    H_alternative = 1 + effect_uplift
    lognormal_stdev = 1 + sample_stdev

    normal_H_null = np.log10(H_null)
    normal_H_alternative = np.log10(H_alternative)
    normal_stdev = np.log10(lognormal_stdev)
    
    test_result_raw = scipy.stats.ttest_ind_from_stats(mean1 = normal_H_null, std1 = normal_stdev,
                                                   nobs1 = np.round(sample_size / 2),
                                                   mean2 = normal_H_alternative, std2 = normal_stdev, 
                                                   nobs2 = np.round(sample_size / 2),
                                                   equal_var = True
                                                   )
    test_result_pvalue = test_result_raw[1]

    test_result_pvalue /= 2
    
    ### graphs
    gridsize = (20, 2)
    fig = plt.figure(figsize = (12, 5))
    ax3 = plt.subplot2grid(gridsize, (0, 0), colspan = 2, rowspan = 1)
    ax1 = plt.subplot2grid(gridsize, (2, 0), colspan = 1, rowspan = gridsize[0] - 1)
    ax2 = plt.subplot2grid(gridsize, (2, 1), colspan = 1, rowspan = gridsize[0] - 1)

    ax3.set_title("Test's pvalue: " + str(np.round(test_result_pvalue, 3)) +
                 "        (Confidence Level: " + str(np.round(100 * (1 - test_result_pvalue), 1)) + "%)")
    ax3.get_xaxis().set_visible(False)
    ax3.get_yaxis().set_visible(False)
    ax3.spines['left'].set_visible(False)
    ax3.spines['right'].set_visible(False)
    ax3.spines['top'].set_visible(False)
    ax3.spines['bottom'].set_visible(False)

    x = np.linspace(scipy.stats.lognorm.ppf(0.01, sample_stdev),
                 scipy.stats.lognorm.ppf(0.99, sample_stdev), 100)
    lognorm_pdf = scipy.stats.lognorm.pdf(x, sample_stdev)
    ax1.plot(x, lognorm_pdf,
             linestyle = '-', color = 'black',
             label = 'Population Distribution')
    ax1.plot([H_alternative, H_alternative], [np.min(lognorm_pdf), np.max(lognorm_pdf)],
             linestyle = '-', color = 'red',
             label = 'Expected Performance')

    ax1.set_xlabel('Performance Shift Away from 100%')
    ax1.set_ylabel('Probability of Performace')
    ax1.set_title('Lognormal Performance Probability')
    ax1.legend(loc='upper right')

    n_per_sample = sample_size / 2
    ### independent 2-sample degrees of freedom
    dof = 2 * n_per_sample - 2 
    x = np.linspace(scipy.stats.t.ppf(0.01, df = dof, scale = normal_stdev),
                     scipy.stats.t.ppf(0.99, df = dof, scale = normal_stdev), 100)
    t_pdf = scipy.stats.t.pdf(x, df = dof, scale = normal_stdev)
    test_statistic = normal_H_alternative / (normal_stdev * np.sqrt(2 / n_per_sample))

    ax2.plot(x, t_pdf,
             linestyle = '-', color = 'black',
             label = 'Population Distribution')
    ax2.plot([test_statistic * normal_stdev for i in range(2)], [np.min(t_pdf), np.max(t_pdf)],
             linestyle = '-', color = 'red',
             label = 'Expected Performance')
    ax2.set_xlabel('Log of Performance shift')
    ax2.set_ylabel('Probability of Performace')
    ax2.set_title('Student-T Transformed Performance Probability')
    ax2.legend(loc='upper right')
    
    return fig, ax1, ax2, ax3

In [23]:
_ = widgets.interact(function_for_interact,
                     effect_uplift = percent_uplift_slider,
                    sample_stdev = st_dev_as_percent_of_mean_slider,
                    sample_size = sample_size_slider)

interactive(children=(FloatSlider(value=0.03, continuous_update=False, description='Expected Percent Uplift:',…

## 2. Sample size needed to get desired confidece given signal strength, test type, variance

In [26]:
seek_p_percent_uplift_slider = widgets.FloatSlider(
    value = 0.03,
    min= 0.0,
    max= 1.0,
    step= 0.01,
    description = 'Expected Percent Uplift:',
    disabled = False,
    continuous_update = False,
    orientation ='horizontal',
    readout = True,
    readout_format = ".0%",
    style = {'description_width': 'initial'},
    layout=dict(width='40%')
)

seek_p_st_dev_as_percent_of_mean_slider = widgets.FloatSlider(
    value = 0.2,
    min= 0.01,
    max= 2.0,
    step= 0.01,
    description = 'St Dev as % of Mean:',
    disabled = False,
    continuous_update = False,
    orientation ='horizontal',
    readout = True,
    readout_format = ".0%",
    style = {'description_width': 'initial'},
    layout=dict(width='60%')
)

seek_p_desired_percent_confidence_slider = widgets.FloatSlider(
    value = 0.95,
    min= 0.1,
    max= 0.99,
    step= 0.01,
    description = 'Desired confidence level:',
    disabled = False,
    continuous_update = False,
    orientation ='horizontal',
    readout = True,
    readout_format = ".0%",
    style = {'description_width': 'initial'},
    layout=dict(width='60%')
)

In [27]:
def function_for_interact_seek_p(effect_uplift, sample_stdev, desired_percent_confidence):
    user_desired_pvalue = 1 - desired_percent_confidence

    H_null = 1
    H_alternative = 1 + effect_uplift
    lognormal_stdev = 1 + sample_stdev

    normal_H_null = np.log10(H_null)
    normal_H_alternative = np.log10(H_alternative)
    normal_stdev = np.log10(lognormal_stdev)
    
    ### optimize
    test_result_pvalue = 1
    n_per_sample = 2
    while test_result_pvalue > user_desired_pvalue and n_per_sample < 5000:
        test_result_raw = scipy.stats.ttest_ind_from_stats(mean1 = normal_H_null, std1 = normal_stdev,
                                                       nobs1 = n_per_sample,
                                                       mean2 = normal_H_alternative, std2 = normal_stdev, 
                                                       nobs2 = n_per_sample,
                                                       equal_var = True
                                                       )
        test_result_pvalue = test_result_raw[1]
    
        test_result_pvalue /= 2
        
        n_per_sample += 1
    
    
    ### graphs
    gridsize = (20, 2)
    fig = plt.figure(figsize = (12, 5))
    ax3 = plt.subplot2grid(gridsize, (0, 0), colspan = 2, rowspan = 1)
    ax1 = plt.subplot2grid(gridsize, (2, 0), colspan = 1, rowspan = gridsize[0] - 1)
    ax2 = plt.subplot2grid(gridsize, (2, 1), colspan = 1, rowspan = gridsize[0] - 1)

    ax3.set_title("Number samples needed per group (half the total study): " + str(n_per_sample))
    ax3.get_xaxis().set_visible(False)
    ax3.get_yaxis().set_visible(False)
    ax3.spines['left'].set_visible(False)
    ax3.spines['right'].set_visible(False)
    ax3.spines['top'].set_visible(False)
    ax3.spines['bottom'].set_visible(False)

    x = np.linspace(scipy.stats.lognorm.ppf(0.01, sample_stdev),
                 scipy.stats.lognorm.ppf(0.99, sample_stdev), 100)
    lognorm_pdf = scipy.stats.lognorm.pdf(x, sample_stdev)
    ax1.plot(x, lognorm_pdf,
             linestyle = '-', color = 'black',
             label = 'Population Distribution')
    ax1.plot([H_alternative, H_alternative], [np.min(lognorm_pdf), np.max(lognorm_pdf)],
             linestyle = '-', color = 'red',
             label = 'Expected Performance')

    ax1.set_xlabel('Performance Shift Away from 100%')
    ax1.set_ylabel('Probability of Performace')
    ax1.set_title('Lognormal Performance Probability')
    ax1.legend(loc='upper right')

    ### independent 2-sample degrees of freedom
    dof = 2 * n_per_sample - 2 
    x = np.linspace(scipy.stats.t.ppf(0.01, df = dof, scale = normal_stdev),
                     scipy.stats.t.ppf(0.99, df = dof, scale = normal_stdev), 100)
    t_pdf = scipy.stats.t.pdf(x, df = dof, scale = normal_stdev)
    test_statistic = normal_H_alternative / (normal_stdev * np.sqrt(2 / n_per_sample))

    ax2.plot(x, t_pdf,
             linestyle = '-', color = 'black',
             label = 'Population Distribution')
    ax2.plot([test_statistic * normal_stdev for i in range(2)], [np.min(t_pdf), np.max(t_pdf)],
             linestyle = '-', color = 'red',
             label = 'Expected Performance')
    ax2.set_xlabel('Log of Performance shift')
    ax2.set_ylabel('Probability of Performace')
    ax2.set_title('Student-T Transformed Performance Probability')
    ax2.legend(loc='upper right')
    
    return fig, ax1, ax2, ax3
    
_ = widgets.interact(function_for_interact_seek_p,
                     effect_uplift = seek_p_percent_uplift_slider,
                    sample_stdev = seek_p_st_dev_as_percent_of_mean_slider,
                    desired_percent_confidence = seek_p_desired_percent_confidence_slider)

interactive(children=(FloatSlider(value=0.03, continuous_update=False, description='Expected Percent Uplift:',…